# SchNet S2EF training example

The purpose of this notebook is to demonstrate some of the basics of the Open Catalyst Project's (OCP) codebase and data. In this example, we will train a schnet model for predicting the energy and forces of a given structure (S2EF task). First, ensure you have installed the OCP ocp repo and all the dependencies according to the [README](https://github.com/Open-Catalyst-Project/ocp/blob/master/README.md).

Disclaimer: This notebook is for tutorial purposes, it is unlikely it will be practical to train baseline models on our larger datasets using this format. As a next step, we recommend trying the command line examples. 

## Imports

In [1]:
import torch
import os
from ocpmodels.trainers import ForcesTrainer
from ocpmodels import models
from ocpmodels.common import logger
from ocpmodels.common.utils import setup_logging
setup_logging()

In [2]:
# a simple sanity check that a GPU is available
if torch.cuda.is_available():
    print("True")
else:
    print("False")

False


## The essential steps for training an OCP model

1) Download data

2) Preprocess data (if necessary)

3) Define or load a configuration (config), which includes the following
   
   - task
   - model
   - optimizer
   - dataset
   - trainer

4) Train

5) Depending on the model/task there might be intermediate relaxation step

6) Predict

## Dataset

This examples uses the LMDB generated from the following [tutorial](http://laikapack.cheme.cmu.edu/notebook/open-catalyst-project/mshuaibi/notebooks/projects/ocp/docs/source/tutorials/lmdb_dataset_creation.ipynb). Please run that notebook before moving on. Alternatively, if you have other LMDBs available you may specify that instead.

In [16]:
# set the path to your local lmdb directory
train_src = "s2ef"

## Define config

For this example, we will explicitly define the config; however, a set of default config files exists in the config folder of this repository. Default config yaml files can easily be loaded with the `build_config` util (found in `ocp/ocpmodels/common/utils.py`). Loading a yaml config is preferrable when launching jobs from the command line. We have included our best models' config files [here](https://github.com/Open-Catalyst-Project/ocp/tree/master/configs/s2ef).

**Task** 

In [17]:
task = {
    'dataset': 'trajectory_lmdb', # dataset used for the S2EF task
    'description': 'Regressing to energies and forces for DFT trajectories from OCP',
    'type': 'regression',
    'metric': 'mae',
    'labels': ['potential energy'],
    'grad_input': 'atomic forces',
    'train_on_free_atoms': True,
    'eval_on_free_atoms': True
}

**Model** - SchNet for this example

In [18]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

**Optimizer**

In [19]:
optimizer = {
    'batch_size': 16, # if hitting GPU memory issues, lower this
    'eval_batch_size': 8,
    'num_workers': 8,
    'lr_initial': 0.0001,
    'scheduler': "ReduceLROnPlateau",
    'mode': "min",
    'factor': 0.8,
    'patience': 3,
    'max_epochs': 80,
    'max_epochs': 1, # used for demonstration purposes
    'force_coefficient': 100,
}

**Dataset**

For simplicity, `train_src` is used for all the train/val/test sets. Feel free to update with the actual S2EF val and test sets, but it does require additional downloads and preprocessing. If you desire to normalize your targets, `normalize_labels` must be set to `True` and corresponding `mean` and `stds` need to be specified. These values have been precomputed for you and can be found in any of the [`base.yml`](https://github.com/Open-Catalyst-Project/ocp/blob/master/configs/s2ef/20M/base.yml#L5-L9) config files.

In [20]:
dataset = [
{'src': train_src, 'normalize_labels': False}, # train set 
{'src': train_src}, # val set (optional)
{'src': train_src} # test set (optional - writes predictions to disk)
]

In [21]:
help(ForcesTrainer)

Help on class ForcesTrainer in module ocpmodels.trainers.forces_trainer:

class ForcesTrainer(ocpmodels.trainers.base_trainer.BaseTrainer)
 |  ForcesTrainer(task, model, dataset, optimizer, identifier, normalizer=None, timestamp_id=None, run_dir=None, is_debug=False, is_hpo=False, print_every=100, seed=None, logger='tensorboard', local_rank=0, amp=False, cpu=False, slurm={}, noddp=False)
 |  
 |  Trainer class for the Structure to Energy & Force (S2EF) and Initial State to
 |  Relaxed State (IS2RS) tasks.
 |  
 |  .. note::
 |  
 |      Examples of configurations for task, model, dataset and optimizer
 |      can be found in `configs/ocp_s2ef <https://github.com/Open-Catalyst-Project/baselines/tree/master/configs/ocp_is2re/>`_
 |      and `configs/ocp_is2rs <https://github.com/Open-Catalyst-Project/baselines/tree/master/configs/ocp_is2rs/>`_.
 |  
 |  Args:
 |      task (dict): Task configuration.
 |      model (dict): Model configuration.
 |      dataset (dict): Dataset configuration.

**Trainer**

Use the `ForcesTrainer` for the S2EF and IS2RS tasks, and the `EnergyTrainer` for the IS2RE task 

In [22]:
trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    # is_vis=False,
    print_every=5,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2023-04-19-20-41-36-SchNet-example
  commit: a8ac36d
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2023-04-19-20-41-36-SchNet-example
  print_every: 5
  results_dir: ./results/2023-04-19-20-41-36-SchNet-example
  seed: 0
  timestamp_id: 2023-04-19-20-41-36-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 0
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
noddp: false
optim:
  batch_size: 16
  eval_batch_size: 8
  factor: 0.8
  force_coefficient: 100
  lr_initial: 0.0001
  max_epochs: 1
  mode: min
  num_workers: 8
  patience: 3
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_atoms: true
 

/home/zwbai/miniconda3/envs/ocp-models/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-04-19 20:41:16 (WARNING): Model gradient logging to tensorboard not yet supported.


## Check the model

In [23]:
print(trainer.model)

OCPDataParallel(
  (module): SchNetWrap(hidden_channels=1024, num_filters=256, num_interactions=3, num_gaussians=200, cutoff=6.0)
)


## Train

In [24]:
trainer.train()

2023-04-19 20:41:23 (INFO): forcesx_mae: 3.20e-01, forcesy_mae: 3.20e-01, forcesz_mae: 3.59e+00, forces_mae: 1.41e+00, forces_cos: -3.49e-03, forces_magnitude: 3.93e+00, energy_mae: 7.04e+00, energy_force_within_threshold: 0.00e+00, loss: 4.30e+02, lr: 1.00e-04, epoch: 7.94e-02, step: 5.00e+00
2023-04-19 20:41:27 (INFO): forcesx_mae: 1.61e-01, forcesy_mae: 1.61e-01, forcesz_mae: 2.13e+00, forces_mae: 8.16e-01, forces_cos: -4.33e-03, forces_magnitude: 2.25e+00, energy_mae: 1.73e+00, energy_force_within_threshold: 0.00e+00, loss: 2.47e+02, lr: 1.00e-04, epoch: 1.59e-01, step: 1.00e+01
2023-04-19 20:41:32 (INFO): forcesx_mae: 7.80e-02, forcesy_mae: 7.78e-02, forcesz_mae: 1.45e+00, forces_mae: 5.34e-01, forces_cos: -7.80e-03, forces_magnitude: 1.50e+00, energy_mae: 1.18e+00, energy_force_within_threshold: 0.00e+00, loss: 1.62e+02, lr: 1.00e-04, epoch: 2.38e-01, step: 1.50e+01
2023-04-19 20:41:36 (INFO): forcesx_mae: 7.42e-02, forcesy_mae: 7.41e-02, forcesz_mae: 8.12e-01, forces_mae: 3.20e-

device 0: 100%|██████████| 126/126 [00:15<00:00,  7.94it/s]

2023-04-19 20:42:17 (INFO): forcesx_mae: 0.0006, forcesy_mae: 0.0005, forcesz_mae: 0.2729, forces_mae: 0.0913, forces_cos: 0.0031, forces_magnitude: 0.2725, energy_mae: 0.3773, energy_force_within_threshold: 0.0000, loss: 27.7826, epoch: 1.0000
2023-04-19 20:42:17 (INFO): Predicting on test.



device 0: 100%|██████████| 126/126 [00:15<00:00,  8.08it/s]

2023-04-19 20:42:33 (INFO): Writing results to ./results/2023-04-19-20-41-36-SchNet-example/s2ef_predictions.npz


### Load Checkpoint
Once training has completed a `Trainer` class, by default, is loaded with the best checkpoint as determined by training or validation (if available) metrics. To load a `Trainer` class directly with a pretrained model, specify the `checkpoint_path` as defined by your previously trained model (`checkpoint_dir`):

In [25]:
checkpoint_path = os.path.join(trainer.config["cmd"]["checkpoint_dir"], "checkpoint.pt")
checkpoint_path

'./checkpoints/2023-04-19-20-41-36-SchNet-example/checkpoint.pt'

In [26]:
model = {
    'name': 'schnet',
    'hidden_channels': 1024, # if training is too slow for example purposes reduce the number of hidden channels
    'num_filters': 256,
    'num_interactions': 3,
    'num_gaussians': 200,
    'cutoff': 6.0
}

pretrained_trainer = ForcesTrainer(
    task=task,
    model=model,
    dataset=dataset,
    optimizer=optimizer,
    identifier="SchNet-example",
    run_dir="./", # directory to save results if is_debug=False. Prediction files are saved here so be careful not to override!
    is_debug=False, # if True, do not save checkpoint, logs, or results
    # is_vis=False,
    print_every=10,
    seed=0, # random seed to use
    logger="tensorboard", # logger of choice (tensorboard and wandb supported)
    local_rank=0,
    amp=False, # use PyTorch Automatic Mixed Precision (faster training and less memory usage)
)

pretrained_trainer.load_checkpoint(checkpoint_path=checkpoint_path)

amp: false
cmd:
  checkpoint_dir: ./checkpoints/2023-04-19-20-41-36-SchNet-example
  commit: a8ac36d
  identifier: SchNet-example
  logs_dir: ./logs/tensorboard/2023-04-19-20-41-36-SchNet-example
  print_every: 10
  results_dir: ./results/2023-04-19-20-41-36-SchNet-example
  seed: 0
  timestamp_id: 2023-04-19-20-41-36-SchNet-example
dataset:
  normalize_labels: false
  src: s2ef
gpus: 0
logger: tensorboard
model: schnet
model_attributes:
  cutoff: 6.0
  hidden_channels: 1024
  num_filters: 256
  num_gaussians: 200
  num_interactions: 3
noddp: false
optim:
  batch_size: 16
  eval_batch_size: 8
  factor: 0.8
  force_coefficient: 100
  lr_initial: 0.0001
  max_epochs: 1
  mode: min
  num_workers: 8
  patience: 3
  scheduler: ReduceLROnPlateau
slurm: {}
task:
  dataset: trajectory_lmdb
  description: Regressing to energies and forces for DFT trajectories from OCP
  eval_on_free_atoms: true
  grad_input: atomic forces
  labels:
  - potential energy
  metric: mae
  train_on_free_atoms: true


/home/zwbai/miniconda3/envs/ocp-models/lib/python3.9/site-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
2023-04-19 20:42:33 (WARNING): Model gradient logging to tensorboard not yet supported.


2023-04-19 20:42:33 (INFO): Loading checkpoint from: ./checkpoints/2023-04-19-20-41-36-SchNet-example/checkpoint.pt


## Predict

If a test has been provided in your config, predictions are generated and written to disk automatically upon training completion. Otherwise, to make predictions on unseen data a `torch.utils.data` DataLoader object must be constructed. Here we reference our test set to make predictions on. Predictions are saved in `{results_file}.npz` in your `results_dir`.

In [27]:
# make predictions on the existing test_loader
predictions = pretrained_trainer.predict(pretrained_trainer.test_loader, results_file="s2ef_results", disable_tqdm=False)

2023-04-19 20:42:33 (INFO): Predicting on test.


device 0: 100%|██████████| 126/126 [00:21<00:00,  5.89it/s]

2023-04-19 20:42:55 (INFO): Writing results to ./results/2023-04-19-20-41-36-SchNet-example/s2ef_s2ef_results.npz


In [28]:
energies = predictions["energy"]
forces = predictions["forces"]